In [28]:
import pandas as pd
import numpy as np
import base64
import string
import re
from collections import Counter
import spacy
from nltk.corpus import stopwords
import random
from spacy.util import minibatch

In [4]:
import nltk
nltk.download('stopwords')


stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/daaa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df = pd.read_csv('ticketClassification.csv')

In [10]:
df.head()

,Unnamed: 0,text,label
0,0,Technical issue Product setup I'm having an is...,3
1,1,Technical issue Peripheral compatibility I'm h...,3
2,2,Technical issue Network problem I'm facing a p...,0
3,3,Billing inquiry Account access I'm having an i...,0
4,4,Billing inquiry Data loss I'm having an issue ...,0


In [41]:
df.text.head(1)

0    Technical issue Product setup I'm having an is...
Name: text, dtype: object

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.33, random_state=42)

In [18]:
df['label'].value_counts() # dataset is balanced :)

label
1    2192
3    2129
2    2085
0    2063
Name: count, dtype: int64

In [24]:
nlp = spacy.load('en_core_web_sm')

In [25]:
def convert_df(df):
    return df.apply(lambda row: (row['text'], {"cats": {row['label']: 1}}), axis=1).tolist()

train_data = convert_df(train)
test_data = convert_df(test)

In [34]:
from spacy.training import Example

def convert_to_example(df):
    examples = []
    for text, annot in df:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annot)
        examples.append(example)
    return examples

train_data = convert_df(train)
train_examples = convert_to_example(train_data)

# Start training
optimizer = nlp.begin_training()

for i in range(10):
    losses = {}
    batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        nlp.update(batch, sgd=optimizer, drop=0.2, losses=losses)
    print("Losses", losses)


Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}
Losses {'textcat': 58.0625}


In [37]:
# Convert the test data to `Example` objects
test_data = convert_df(test)
test_examples = convert_to_example(test_data)

# Use the model to predict the test data
correct_predictions = 0
total_predictions = 0

for example in test_examples:
    # Use the model to predict the text
    doc = nlp(example.text)
    predicted_label = max(doc.cats, key=doc.cats.get)  # Get the label with the highest score

    # Get the actual label
    actual_label = max(example.cats, key=example.annotation.cats.get)
    
    # If the predicted label matches the actual label, increment correct_predictions
    if predicted_label == actual_label:
        correct_predictions += 1

    total_predictions += 1

# Print the accuracy
print(f"Accuracy: {correct_predictions/total_predictions:.2f}")


AttributeError: 'spacy.training.example.Example' object has no attribute 'cats'

In [52]:
from sklearn.metrics import accuracy_score

# Prepare the test data
test_data = convert_df(test)
test_examples = convert_to_example(test_data)

# Get the true labels
true_labels = [max(example.y.cats, key=example.y.cats.get) for example in test_examples]

# Predict the labels with the model
predicted_labels = []
for example in test_examples:
    doc = nlp(example.text)
    predicted_labels.append(max(doc.cats, key=doc.cats.get))

# Ensure the labels are in the same format
true_labels = [str(label) for label in true_labels]
predicted_labels = [str(label) for label in predicted_labels]

# Calculate the accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

print(f'Test Accuracy: {accuracy*100:.2f}%')




Test Accuracy: 0.00%


In [53]:
print("True labels:", true_labels[:10]) 
print("Predicted labels:", predicted_labels[:10])


True labels: ['2', '0', '0', '2', '1', '0', '1', '2', '2', '0']
Predicted labels: ['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low']


In [1]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer


/Users/daaa/Downloads/KAKI-App/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
dataset = load_dataset('csv', data_files="ticketClassification.csv", split='train')

Found cached dataset csv (/Users/daaa/.cache/huggingface/datasets/csv/default-3ccb1e3336d246a2/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


In [13]:
dataset

Dataset({
    features: ['Unnamed: 0', 'text', 'label'],
    num_rows: 8469
})

In [16]:
stratify_column_name = "label"

dataset.class_encode_column(
    stratify_column_name
).train_test_split(
    test_size=0.2, 
    stratify_by_column=stratify_column_name
)


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 6775
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 1694
    })
})

In [22]:
# Select N examples per class (8 in this case)
# train_ds = dataset["train"]
# test_ds = dataset["test"]

split_data = dataset.train_test_split(test_size=0.2)

In [23]:
train_ds = split_data['train']
test_ds = split_data['test']

In [25]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [26]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

Generating Training Pairs: 100%|██████████| 20/20 [00:02<00:00,  8.40it/s]
***** Running training *****
  Num examples = 271000
  Num epochs = 1
  Total optimization steps = 16938
  Total train batch size = 16
Epoch:   0%|          | 0/1 [25:04<?, ?it/s]


KeyboardInterrupt: 